In [45]:
from datetime import datetime

import pandas as pd

In [2]:
def parse(x):
    return datetime.strptime(x, "%Y %m %d %H")

In [3]:
dataset = pd.read_csv(
    "pollution.csv",
    parse_dates=[["year", "month", "day", "hour"]],
    index_col=0,
    date_parser=parse,
)

In [4]:
dataset.drop("No", axis=1, inplace=True)

In [5]:
dataset.columns = [
    "pollution",
    "dew",
    "temp",
    "press",
    "wnd_dir",
    "wnd_spd",
    "snow",
    "rain",
]
dataset.index.name = "date"
# mark all NA values with 0
dataset["pollution"].fillna(0, inplace=True)
# drop the first 24 hours
dataset = dataset[24:]

In [6]:
dataset.head()

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0


In [7]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [8]:
encoder = LabelEncoder()
values = dataset.values

In [9]:
values[:, 4] = encoder.fit_transform(values[:, 4])

In [10]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [11]:
scaled = scaler.fit_transform(values)

In [12]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [("var%d(t-%d)" % (j + 1, i)) for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [("var%d(t)" % (j + 1)) for j in range(n_vars)]
        else:
            names += [("var%d(t+%d)" % (j + 1, i)) for j in range(n_vars)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [13]:
reframed = series_to_supervised(scaled, 1, 1)

# dropping cols we dont want to predict
reframed.drop(reframed.columns[[9, 10, 11, 12, 13, 14, 15]], axis=1, inplace=True)

In [14]:
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var1(t)
1,0.129779,0.352941,0.245902,0.527273,0.666667,0.002290,0.000000,0.0,0.148893
2,0.148893,0.367647,0.245902,0.527273,0.666667,0.003811,0.000000,0.0,0.159960
3,0.159960,0.426471,0.229508,0.545455,0.666667,0.005332,0.000000,0.0,0.182093
4,0.182093,0.485294,0.229508,0.563636,0.666667,0.008391,0.037037,0.0,0.138833
5,0.138833,0.485294,0.229508,0.563636,0.666667,0.009912,0.074074,0.0,0.109658


In [15]:
n_train_hours = 365 * 24
train = reframed[:n_train_hours]
test = reframed[n_train_hours:]

In [16]:
train.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var1(t)
1,0.129779,0.352941,0.245902,0.527273,0.666667,0.002290,0.000000,0.0,0.148893
2,0.148893,0.367647,0.245902,0.527273,0.666667,0.003811,0.000000,0.0,0.159960
3,0.159960,0.426471,0.229508,0.545455,0.666667,0.005332,0.000000,0.0,0.182093
4,0.182093,0.485294,0.229508,0.563636,0.666667,0.008391,0.037037,0.0,0.138833
5,0.138833,0.485294,0.229508,0.563636,0.666667,0.009912,0.074074,0.0,0.109658


In [24]:
X = train.drop("var1(t)", axis=1)
y = train["var1(t)"]

In [26]:
X.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1)
1,0.129779,0.352941,0.245902,0.527273,0.666667,0.002290,0.000000,0.0
2,0.148893,0.367647,0.245902,0.527273,0.666667,0.003811,0.000000,0.0
3,0.159960,0.426471,0.229508,0.545455,0.666667,0.005332,0.000000,0.0
4,0.182093,0.485294,0.229508,0.563636,0.666667,0.008391,0.037037,0.0
5,0.138833,0.485294,0.229508,0.563636,0.666667,0.009912,0.074074,0.0


In [28]:
y.head()

1    0.148893
2    0.159960
3    0.182093
4    0.138833
5    0.109658
Name: var1(t), dtype: float64

In [29]:
import numpy as np


def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [30]:
X = [10, 20, 30, 40, 50, 60, 70, 80, 90]

In [31]:
n_steps = 3

In [32]:
X, y = split_sequence(X, n_steps)

In [33]:
X

array([[10, 20, 30],
       [20, 30, 40],
       [30, 40, 50],
       [40, 50, 60],
       [50, 60, 70],
       [60, 70, 80]])

In [34]:
y

array([40, 50, 60, 70, 80, 90])

In [36]:
from tensorflow import keras

In [37]:
# define model
model = keras.models.Sequential()
model.add(keras.layers.Dense(100, activation="relu", input_dim=n_steps))
model.add(keras.layers.Dense(1))
model.compile(optimizer="adam", loss="mse")

2022-10-17 10:45:34.174506: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /app/lib
2022-10-17 10:45:34.174545: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-17 10:45:34.174573: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-10-17 10:45:34.178804: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [42]:
model.fit(X, y, epochs=200)

Epoch 1/200
1/1 [==============================] - 0s 7ms/step - loss: 1937.0470
Epoch 2/200
1/1 [==============================] - 0s 7ms/step - loss: 1830.4032
Epoch 3/200
1/1 [==============================] - 0s 5ms/step - loss: 1727.9540
Epoch 4/200
1/1 [==============================] - 0s 6ms/step - loss: 1629.8126
Epoch 5/200
1/1 [==============================] - 0s 6ms/step - loss: 1535.2643
Epoch 6/200
1/1 [==============================] - 0s 8ms/step - loss: 1443.7164
Epoch 7/200
1/1 [==============================] - 0s 7ms/step - loss: 1355.5659
Epoch 8/200
1/1 [==============================] - 0s 7ms/step - loss: 1270.8661
Epoch 9/200
1/1 [==============================] - 0s 5ms/step - loss: 1189.5439
Epoch 10/200
1/1 [==============================] - 0s 6ms/step - loss: 1111.7380
Epoch 11/200
1/1 [==============================] - 0s 7ms/step - loss: 1037.5265
Epoch 12/200
1/1 [==============================] - 0s 6ms/step - loss: 966.6490
Epoch 13/200
1/1 [========

In [43]:
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps))
x_input.shape

(1, 3)

In [44]:
model.predict(x_input)

1/1 [==============================] - 0s 20ms/step


array([[107.75781]], dtype=float32)